In [1]:
import covamd
import numpy as np

In [2]:
class Potential(object):
    """
    Base class for all MD potentials
    """
    def __init__(self, name: str):
        self.name = name

    def __call__(self, r: float):
        raise NotImplementedError

In [3]:
class LennardJones(Potential):
    """
    Lennard Jones 12-6 potential class
    Input(s)
    epsilon = interaction strength
    sigma = potential interaction length scale
    cutoff = distance beyond which the potential is zero
    """
    def __init__(self, epsilon: float, sigma: float, cutoff: float = 2.5, name = "lj12/6"):
        self.epsilon = epsilon
        self.sigma = sigma
        self.cutoff = cutoff
        super().__init__(name=name)

    def __call__(self, r: float):
        """
        Compute the potential value given the distance `r`.
        The potential is set to 0 if r > cutoff
        """
        if r <= self.cutoff:
            return 4*self.epsilon*((self.sigma/r)**12 - (self.sigma/r)**6)
        return 0.0
        

In [4]:
class Domain(object):
    def __init__(self, xlo: float, xhi: float, ylo: float, yhi: float):
        self.xlo = xlo
        self.xhi = xhi
        self.ylo = ylo
        self.yhi = yhi


    @property
    def lx(self):
        return self.xhi - self.xlo

    @property
    def ly(self):
        return self.yhi - self.ylo

In [5]:
class Particle(object):
    def __init__(self, id: int, mass: float, x: float, y: float,
                 vx: float = 0.0, vy: float = 0.0, fx: float = 0.0,
                 fy: float = 0.0):
        """
        Basic particle
        """
        self.id = id
        self.mass = mass
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.fx = fx
        self.fy = fy

    def __eq__(self, other):
        return all([other.__dict__[key] == value for key, value in
                   self.__dict__.items()])

    def __sub__(self, other):
        return self.x - other.x, self.y - other.y

    def __repr__(self):
        return f"Particle(id={self.id}, mass={self.mass},x={self.x},\
                y={self.y},vx={self.vx},vy={self.vy},\
                fx={self.fx},fy={self.fy})".replace('\n', '')

    def __str__(self):
        return f"""{self.id},{self.mass},{self.x},{self.y},{self.vx},{self.vy},{self.fx},{self.fy}\n"""

def create_particle(id: int, mass: float, domain: Domain):
    """
    Create a particle with id=`id` and mass=`mass' randomly
    located in the domain
    """
    return Particle(id=id, mass=mass,
                    x=np.random.uniform(domain.xlo, domain.xhi),
                    y=np.random.uniform(domain.ylo, domain.yhi))

In [6]:
domain = Domain(xlo = 0, xhi = 20, ylo = 0, yhi = 20)
num_particles = 30
nsteps = 1000
particles = [create_particle(id=index, mass=1.0, domain = domain) for index in range(num_particles)]

In [7]:
lj = LennardJones(epsilon=1.0, sigma=1.0)

In [8]:
#covamd.total_pe(particles, lj, domain)

In [ ]:
particles[0].x